In [21]:
import pandas as pd

# Read the Excel files into DataFrames
staff_df = pd.read_excel("staff.xlsx")
types_df = pd.read_excel("types.xlsx")
departments_df = pd.read_excel("departments.xlsx")

# Merge the DataFrames
merged_df = staff_df.merge(types_df, on="Project", how="left")
merged_df = merged_df.merge(departments_df, on="Job", how="left")

# Save the merged DataFrame to an Excel file
merged_df.to_excel("all.xlsx", index=False)

print("Data merged and saved to 'all.xlsx' successfully!")

Data merged and saved to 'all.xlsx' successfully!


In [22]:
# Read the merged data from "all.xlsx"
df = pd.read_excel("all.xlsx")

# Group by project, job title, and type, counting occurrences in each month
grouped_df = df.groupby(["Project", "Job", "Type"])["Month"].value_counts().unstack(fill_value=0)

# Find the peak number for each job title within each project-type combination
peak_counts = grouped_df.max(axis=1)

# Reset the index to include project, job title, and type
result = peak_counts.reset_index()

# Rename the column before saving
result = peak_counts.reset_index().rename(columns={0: "Peak Count"})

# Save the result with the Peak Count column**
result.to_excel("unique_job_titles_peak_counts_with_types.xlsx", index=False)

In [23]:
# Read the data
df = pd.read_excel("unique_job_titles_peak_counts_with_types.xlsx")

# Group by type and job, calculate average peak count
grouped_df = df.groupby(["Type", "Job"])["Peak Count"].mean().round().astype(int).reset_index()

# Create the final DataFrame
final_df = pd.DataFrame()
for type in grouped_df["Type"].unique():
    type_data = grouped_df[grouped_df["Type"] == type]
    final_df = pd.concat([final_df, type_data.set_index("Job").rename_axis(type)], axis=1)  # Removed `.to_frame()`

# Save to Excel with a single sheet
with pd.ExcelWriter("example_projects_structured.xlsx") as writer:
    final_df.to_excel(writer, sheet_name="Example Projects", index=True)

print("Example projects saved to 'example_projects_structured.xlsx' successfully!")

Example projects saved to 'example_projects_structured.xlsx' successfully!


In [38]:
import pandas as pd
import numpy as np
from scipy.stats import norm

# Read the data
df = pd.read_excel("unique_job_titles_peak_counts_with_types.xlsx")

# Get user input for type and duration
user_type = input("Enter the type: ")
user_duration = int(input("Enter the duration in months: "))


In [41]:

# Filter data based on user input
filtered_df = df[df["Type"] == user_type]

# Set the job title as the index for easier access
filtered_df = filtered_df.set_index("Job")

# Get the number of job titles
num_job_titles = len(filtered_df)

# Create a DataFrame for the specified duration
months = pd.date_range(start="2024-01-01", periods=user_duration, freq="MS")
result_df = pd.DataFrame(index=filtered_df.index, columns=months)

# Generate counts in a bell curve pattern for each job title
for job_title in filtered_df.index:
    mean = user_duration / 2  # Peak in the middle
    counts = norm.pdf(np.arange(1, user_duration + 1), loc=mean, scale=mean / 3) * filtered_df.at[job_title, "Peak Count"].values
    counts = np.round(counts).astype(int)
    result_df.loc[job_title, :] = counts[:, np.newaxis]  # Add a new axis

# Save the result to an Excel file
result_df.to_excel("job_titles_simulation.xlsx", index=True)

# Save the result to an Excel file
result_df.to_excel("job_titles_simulation.xlsx", index=True)

# Transpose the DataFrame for a better layout
result_df = result_df.transpose()

# Save to Excel
output_filename = f"{user_type}_job_titles_generated.xlsx"
result_df.to_excel(output_filename, index=True)

print(f"Generated job titles for type '{user_type}' saved to '{output_filename}' successfully!")


ValueError: operands could not be broadcast together with shapes (12,) (2,) 